In [1]:
!curl -s -X DELETE http://localhost:9200/sqe_metrics_sample_data/

{"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [sqe_metrics_sample_data]","index":"sqe_metrics_sample_data","resource.id":"sqe_metrics_sample_data","resource.type":"index_or_alias","index_uuid":"_na_"}],"type":"index_not_found_exception","reason":"no such index [sqe_metrics_sample_data]","index":"sqe_metrics_sample_data","resource.id":"sqe_metrics_sample_data","resource.type":"index_or_alias","index_uuid":"_na_"},"status":404}

In [2]:
# Hack needed due to not high enough disk space locally
# See reference at bottom of Notebook

# !curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_cluster/settings -d '{ "transient": { "cluster.routing.allocation.disk.threshold_enabled": false } }'

In [3]:
import time
import json
import random
import uuid
import pandas as pd

from opensearchpy import OpenSearch

client = OpenSearch("http://localhost:9200", use_ssl=False)

In [4]:
df = pd.read_csv("metrics.csv")

In [5]:
df.head()

,timestamp,search_config,query_set_id,query,metric,value,application,evaluation_id
0,2024-07-01,research_1,sampled_queries,$30 roblox gift card not digital,dcg,0.000000,sample_data,research_1_2024-07-01 00:00:00
1,2024-07-01,research_1,sampled_queries,(fiction without frontiers),dcg,0.000000,sample_data,research_1_2024-07-01 00:00:00
2,2024-07-01,research_1,sampled_queries,100,dcg,1.630930,sample_data,research_1_2024-07-01 00:00:00
3,2024-07-01,research_1,sampled_queries,15 inch light weight laptop that has lots of m...,dcg,1.045284,sample_data,research_1_2024-07-01 00:00:00
4,2024-07-01,research_1,sampled_queries,15 microsoft card,dcg,0.000000,sample_data,research_1_2024-07-01 00:00:00


In [6]:
bulk = []

for ix, metric in df.iterrows():
    metric_dict = {
        "timestamp": metric["timestamp"],
        "search_config": metric["search_config"],
        "query_set_id": metric["query_set_id"],
        "query": metric["query"],
        "metric": metric["metric"],
        "value": metric["value"],
        "application": metric["application"],
        "evaluation_id": metric["evaluation_id"],
    }
    event_id = str(uuid.uuid4())
    bulk.append(
        {"index": {"_index": "sqe_metrics_sample_data", "_id": event_id}}
    )
    bulk.append(metric_dict)

response = client.bulk(bulk)

In [7]:
df.query_set_id.size

36960

In [8]:
!curl -s http://localhost:9200/sqe_metrics_sample_data/_count | jq

{
  "count": 36960,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  }
}


In [9]:
!curl -s -XGET http://localhost:9200/sqe_metrics_sample_data/_search -H 'Content-Type: application/json' -d' \
{ \
  "query": { \
    "match": { \
      "query": "4t camo sweat shirt" \
    } \
  } \
}' | jq


{
  "took": 39,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 336,
      "relation": "eq"
    },
    "max_score": 4.5289173,
    "hits": [
      {
        "_index": "sqe_metrics_sample_data",
        "_id": "06190b7d-ac0f-40f2-9d4f-760efd0e9627",
        "_score": 4.5289173,
        "_source": {
          "timestamp": "2024-07-01",
          "search_config": "research_1",
          "query_set_id": "top_queries",
          "query": "plus size black shirt",
          "metric": "dcg",
          "value": 2.9484591188793923,
          "application": "sample_data",
          "evaluation_id": "research_1_2024-07-01 00:00:00"
        }
      },
      {
        "_index": "sqe_metrics_sample_data",
        "_id": "b6ed6d06-67a4-48b2-be0c-7159441f17bb",
        "_score": 4.5289173,
        "_source": {
          "timestamp": "2024-06-30",
          "search_config": "research_1",
          

In [10]:
# Hacky, disk space checks need to be disabled on my system to allow Dashboards to work
#
# https://stackoverflow.com/a/57675654

# !curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_cluster/settings -d '{ "transient": { "cluster.routing.allocation.disk.threshold_enabled": false } }'
# !curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_all/_settings -d '{"index.blocks.read_only_allow_delete": null}'